In [1]:
import nltk
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [2]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [6]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec

import gensim.downloader as api
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import KeyedVectors

from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import numpy as np

from tensorflow.keras import regularizers



## **single layer model with trainable = false**

In [8]:
# Define ModelCheckpoint callback to save the model in .keras format
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="model_checkpoint.keras",
                                                         save_best_only=True,
                                                         save_weights_only=False)

# EarlyStopping callback to stop training when validation accuracy stops improving
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    verbose=1
)


In [43]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras import Model, Input

# Parameters
# ==================================================
# Data loading params
dev_sample_percentage = 0.1  # Percentage of the training data to use for validation
positive_data_file = "rt-polarity.pos"  # Path to the positive data file
negative_data_file = "rt-polarity.neg"  # Path to the negative data file

# Model Hyperparameters
embedding_dim = 100  # Adjusted embedding dimension to match Word2Vec
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

# Misc Parameters
allow_soft_placement = True
log_device_placement = False

# Data Preparation
# ==================================================

# Load data
print("Loading data...")
positive_examples = list(open(positive_data_file, "r").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_data_file, "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
y = [1 if i < len(positive_examples) else 0 for i in range(len(x_text))]
X_train, X_test, y_train, y_test = train_test_split(x_text, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")

# Tokenization
sentences = [word_tokenize(sentence) for sentence in X_train]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

# Train the Word2Vec model
w2v_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=5, workers=4)

# Create the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

def create_text_cnn():
    input_layer = Input(shape=(max_length,))
    embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)(input_layer)
    conv_layer = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, padding='valid', activation='relu')(embedding_layer)
    maxpool_layer = tf.keras.layers.MaxPooling1D(pool_size=max_length - 3 + 1)(conv_layer)
    flatten_layer = tf.keras.layers.Flatten()(maxpool_layer)
    output_layer = tf.keras.layers.Dense(2, activation='softmax')(flatten_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create and compile the model
textcnn_model = create_text_cnn()
textcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Ensure the model is built by calling it on dummy data
dummy_input = np.zeros((1, max_length))  # Dummy input with shape (1, max_length)
textcnn_model(dummy_input)  # Call the model to build it

# Train the TextCNN model
history = textcnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=num_epochs
)

# Ensure model training is completed before feature extraction
print("Training complete.")

# Create a feature extraction model to get the outputs of the Flatten layer
feature_extraction_model = Model(inputs=textcnn_model.input, outputs=textcnn_model.layers[-2].output)

# Extract features from the Flatten layer
X_train_features = feature_extraction_model.predict(X_train)
X_test_features = feature_extraction_model.predict(X_test)

# Train logistic regression model on the extracted features
logreg_model = LogisticRegression(penalty='l2', C=0.8)
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Loading data...
Training samples: 8635, Validation samples: 960, Test samples: 1067
Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


135/135 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5402 - loss: 0.6953 - val_accuracy: 0.5750 - val_loss: 0.6804
Epoch 2/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5637 - loss: 0.6811 - val_accuracy: 0.5531 - val_loss: 0.6825
Epoch 3/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5646 - loss: 0.6814 - val_accuracy: 0.5417 - val_loss: 0.6938
Epoch 4/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5628 - loss: 0.6770 - val_accuracy: 0.5552 - val_loss: 0.6830
Epoch 5/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5816 - loss: 0.6743 - val_accuracy: 0.5792 - val_loss: 0.6742
Epoch 6/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5637 - loss: 0.6816 - val_accuracy: 0.5479 - val_loss: 0.6892
Epoch 7/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5917 - loss: 0.6706 - val_accuracy: 0.5573 - val_loss: 0.6798
Epoch 8/200
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5929 - loss: 0.6687 - val_accuracy: 0.583

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
!ls /content/


drive  rt-polarity.neg	rt-polarity.pos  sample_data


In [45]:
# Train logistic regression model on the extracted features
logreg_model = LogisticRegression()
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

accuracy2 = logreg_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy2:.4f}")

Test Accuracy: 0.5998
Train Accuracy: 0.7730


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=85, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)
random_forest_model.fit(X_train_features, y_train)

# Evaluate Random Forest model
accuracy = random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
accuracy3 = random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy3:.4f}")

Test Accuracy: 0.5951
Train Accuracy: 0.9991


In [47]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5961
Train Accuracy: 0.7778


# AS svm is working betterly so testing it on realtime human input

In [49]:
# Manual sentence input and prediction
def predict_sentence(sentence):
    # Preprocess the input sentence
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])
    padded_sentence = pad_sequences(tokenized_sentence, maxlen=max_length, padding='post')

    # Extract features using the feature extraction model
    sentence_features = feature_extraction_model.predict(padded_sentence)  # Only one input

    # Predict using the SVM model
    prediction = svm_model.predict(sentence_features)

    return "Positive" if prediction[0] == 1 else "Negative"

# Example usage: input your sentence here
input_sentence = input("Enter a sentence for sentiment prediction: ")
result = predict_sentence(input_sentence)
print(f"Sentiment Prediction: {result}")


Enter a sentence for sentiment prediction: i m bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Sentiment Prediction: Negative


# 2 layer structure : 1. trainable=false 2. trainable=true

In [36]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = 0.1  # Percentage of the training data to use for validation
positive_data_file = "rt-polarity.pos"
negative_data_file = "rt-polarity.neg"

# Model Hyperparameters
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size = 64
num_epochs = 19
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

# Misc Parameters
allow_soft_placement = True
log_device_placement = False

# Data Preparation
# ==================================================

# Load data
print("Loading data...")
positive_examples = list(open(positive_data_file, "r").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_data_file, "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
y = [1 if i < len(positive_examples) else 0 for i in range(len(x_text))]
X_train, X_test, y_train, y_test = train_test_split(x_text, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(len(X_train), len(X_test), len(y_train), len(y_test))

sentences = [word_tokenize(sentence) for sentence in X_train]
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
print("adhi", len(X_train))
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
print("adhi1", len(X_train))

# Train the Word2Vec model
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

def create_text_cnn():
    # Define inputs for each channel
    input_static = tf.keras.layers.Input(shape=(max_length,))
    input_non_static = tf.keras.layers.Input(shape=(max_length,))

    # Static channel: Use pre-trained embedding matrix and set trainable=False
    embedding_static = tf.keras.layers.Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], input_length=max_length, trainable=False)(input_static)

    # Non-static channel: Use separate embedding layer with trainable=True
    embedding_non_static = tf.keras.layers.Embedding(vocab_size, embedding_matrix.shape[1], input_length=max_length, trainable=True)(input_non_static)

    # Convolution and max-pooling layers for both channels
    conv_static = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, padding='valid', activation='relu')(embedding_static)
    conv_non_static = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, padding='valid', activation='relu')(embedding_non_static)

    maxpool_static = tf.keras.layers.GlobalMaxPooling1D()(conv_static)
    maxpool_non_static = tf.keras.layers.GlobalMaxPooling1D()(conv_non_static)

    # Concatenate the outputs of both channels
    merged = tf.keras.layers.Concatenate()([maxpool_static, maxpool_non_static])

    # Dropout layer for regularization
    dropout = tf.keras.layers.Dropout(0.7)(merged)

    # Output layer
    output = tf.keras.layers.Dense(2, activation='softmax')(dropout)

    # Define the model with two inputs and one output
    model = tf.keras.Model(inputs=[input_static, input_non_static], outputs=output)

    return model

textcnn_model = create_text_cnn()
textcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Train the TextCNN model without early stopping
history = textcnn_model.fit([X_train, X_train], y_train,
                            validation_data=([X_val, X_val], y_val),
                            batch_size=batch_size,
                            epochs=num_epochs)

# Remove the last layers of the TextCNN model
for layer in textcnn_model.layers[:-3]:
    layer.trainable = False

from tensorflow.keras.models import Model

# Define a new model that outputs the Flatten layer's activations
feature_extraction_model = Model(inputs=textcnn_model.input, outputs=textcnn_model.layers[-2].output)

# Extract features from the Flatten layer
X_train_features = feature_extraction_model.predict([X_train, X_train])
X_test_features = feature_extraction_model.predict([X_test, X_test])

print(X_train_features)


Loading data...
8635 1067 8635 1067
adhi 8635
adhi1 8635
Epoch 1/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.5074 - loss: 0.7275 - val_accuracy: 0.6260 - val_loss: 0.6680
Epoch 2/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6658 - loss: 0.6313 - val_accuracy: 0.7417 - val_loss: 0.5413
Epoch 3/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8354 - loss: 0.4113 - val_accuracy: 0.7719 - val_loss: 0.4796
Epoch 4/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9173 - loss: 0.2426 - val_accuracy: 0.7750 - val_loss: 0.5485
Epoch 5/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9634 - loss: 0.1247 - val_accuracy: 0.7542 - val_loss: 0.6760
Epoch 6/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9821 - loss: 0.0667 - val_accuracy: 0.7521 - val_loss: 0.7793
Epoch 7/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9876 - loss: 0.0439 - val_accuracy: 0.7500 - val_loss: 0.8944
Epoch 8/19
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms

In [ ]:
len(X_train)

8635

In [37]:
# Train logistic regression model on the extracted features
logreg_model = LogisticRegression()
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

accuracy2 = logreg_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy2:.4f}")

Test Accuracy: 0.7104
Train Accuracy: 1.0000


In [38]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=90,max_depth=5, min_samples_split=4, min_samples_leaf=2, max_features='sqrt', random_state=42)
random_forest_model.fit(X_train_features, y_train)

# Evaluate Random Forest model
accuracy = random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
accuracy3 = random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy3:.4f}")

Test Accuracy: 0.7048
Train Accuracy: 1.0000


In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='linear', C=0.7,gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='rbf',gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Instantiate the SVM classifier
svm_model = SVC(random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_features, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Instantiate the SVM model with the best parameters
best_svm_model = SVC(**best_params, random_state=42)

# Fit the model to the training data
best_svm_model.fit(X_train_features, y_train)

# Evaluate the model on the test data
accuracy_test = best_svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy_test:.4f}")

# Evaluate the model on the training data
accuracy_train = best_svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy_train:.4f}")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize kNN classifier
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train kNN model
knn_model.fit(X_train_features, y_train)

# Evaluate kNN model
accuracy_test = knn_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy_test:.4f}")

# Evaluate kNN model on the training data
accuracy_train = knn_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy_train:.4f}")


# **1 layer with trainable=true**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras import Model, Input

# Parameters
# ==================================================
# Data loading params
dev_sample_percentage = 0.1  # Percentage of the training data to use for validation
positive_data_file = "rt-polarity.pos"  # Path to the positive data file
negative_data_file = "rt-polarity.neg"  # Path to the negative data file

# Model Hyperparameters
embedding_dim = 100  # Adjusted embedding dimension to match Word2Vec
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size = 64
num_epochs = 200
evaluate_every = 100
checkpoint_every = 100
num_checkpoints = 5

# Misc Parameters
allow_soft_placement = True
log_device_placement = False

# Data Preparation
# ==================================================

# Load data
print("Loading data...")
positive_examples = list(open(positive_data_file, "r").readlines())
positive_examples = [s.strip() for s in positive_examples]
negative_examples = list(open(negative_data_file, "r").readlines())
negative_examples = [s.strip() for s in negative_examples]
x_text = positive_examples + negative_examples
y = [1 if i < len(positive_examples) else 0 for i in range(len(x_text))]
X_train, X_test, y_train, y_test = train_test_split(x_text, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")

# Tokenization
sentences = [word_tokenize(sentence) for sentence in X_train]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

# Train the Word2Vec model
w2v_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=5, workers=4)

# Create the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

def create_text_cnn():
    input_layer = Input(shape=(max_length,))
    embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=True)(input_layer)
    conv_layer = tf.keras.layers.Conv1D(filters=num_filters, kernel_size=3, padding='valid', activation='relu')(embedding_layer)
    maxpool_layer = tf.keras.layers.MaxPooling1D(pool_size=max_length - 3 + 1)(conv_layer)
    flatten_layer = tf.keras.layers.Flatten()(maxpool_layer)
    output_layer = tf.keras.layers.Dense(2, activation='softmax')(flatten_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create and compile the model
textcnn_model = create_text_cnn()
textcnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Ensure the model is built by calling it on dummy data
dummy_input = np.zeros((1, max_length))  # Dummy input with shape (1, max_length)
textcnn_model(dummy_input)  # Call the model to build it

# Train the TextCNN model
history = textcnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=num_epochs
)

# Ensure model training is completed before feature extraction
print("Training complete.")

# Create a feature extraction model to get the outputs of the Flatten layer
feature_extraction_model = Model(inputs=textcnn_model.input, outputs=textcnn_model.layers[-2].output)

# Extract features from the Flatten layer
X_train_features = feature_extraction_model.predict(X_train)
X_test_features = feature_extraction_model.predict(X_test)

# Train logistic regression model on the extracted features
logreg_model = LogisticRegression(penalty='l2', C=0.8)
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# Train logistic regression model on the extracted features
logreg_model = LogisticRegression()
logreg_model.fit(X_train_features, y_train)

# Evaluate logistic regression model
accuracy = logreg_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

accuracy2 = logreg_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy2:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=85, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', random_state=42)
random_forest_model.fit(X_train_features, y_train)

# Evaluate Random Forest model
accuracy = random_forest_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
accuracy3 = random_forest_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy3:.4f}")

In [ ]:
from sklearn.svm import SVC

# Initialize SVM model with regularization parameters
svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)

# Train SVM model
svm_model.fit(X_train_features, y_train)

# Evaluate SVM model
accuracy = svm_model.score(X_test_features, y_test)
print(f"Test Accuracy: {accuracy:.4f}")
# Evaluate SVM model
accuracy = svm_model.score(X_train_features, y_train)
print(f"Train Accuracy: {accuracy:.4f}")